In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# 1. Configurazione Dati Statici (Prezzi pacchetto)
prezzi_pacco = {
    2000: 2.05, 2001: 2.15, 2002: 2.76, 2003: 3.12, 2004: 3.28,
    2005: 3.48, 2006: 3.65, 2007: 3.80, 2008: 3.86, 2009: 4.02,
    2010: 4.11, 2011: 4.25, 2012: 4.35, 2013: 4.45, 2014: 4.51,
    2015: 4.57, 2016: 4.68, 2017: 4.74, 2018: 4.79, 2019: 4.86,
    2020: 4.90, 2021: 4.93, 2022: 4.98, 2023: 5.12, 2024: 5.38,
    2025: 5.68
}

path = "./datasets/consumo_medio.csv"

def generate_dual_axis_chart():
    if not os.path.exists(path):
        print(f"ERRORE: Il file '{path}' non è stato trovato.")
        return

    try:
        # 2. Caricamento e preparazione dati
        df = pd.read_csv(path)

        df_it = df[
            (df["Territorio"] == "Italia") &
            (df["Indicatore"].str.contains("Numero medio di sigarette", na=False))
        ].copy()

        consumo = df_it[["TIME_PERIOD", "Osservazione"]].copy()
        consumo.columns = ["Anno", "Sigarette_medie_giornaliere_per_100_persone"]
        consumo["Anno"] = pd.to_numeric(consumo["Anno"], errors='coerce')
        consumo = consumo.dropna(subset=["Anno"]).sort_values("Anno")

        # 3. Calcoli (Prezzo e Spesa)
        # Usiamo .get(x, 5.0) come nel tuo script per fallback, ma mappiamo prima
        consumo["Prezzo_Pacco"] = consumo["Anno"].map(lambda x: prezzi_pacco.get(x, 5.0))
        consumo["Prezzo_sigaretta"] = consumo["Prezzo_Pacco"] / 20

        consumo["Spesa_media_giornaliera_per_100"] = (
            consumo["Sigarette_medie_giornaliere_per_100_persone"] * consumo["Prezzo_sigaretta"]
        ).round(2)

        # 4. Creazione Grafico Doppio Asse con Plotly
        # Creiamo una figura con un asse Y secondario
        fig = make_subplots(specs=[[{"secondary_y": True}]])

        # Traccia 1: Consumo (Asse Y sinistro - Blu)
        fig.add_trace(
            go.Scatter(
                x=consumo["Anno"],
                y=consumo["Sigarette_medie_giornaliere_per_100_persone"],
                name="Consumo medio (n°)",
                mode='lines+markers',
                marker=dict(symbol='circle'), # Corrisponde a marker='o'
                line=dict(color='#1f77b4', width=3) # 'tab:blue'
            ),
            secondary_y=False
        )

        # Traccia 2: Spesa (Asse Y destro - Rosso)
        fig.add_trace(
            go.Scatter(
                x=consumo["Anno"],
                y=consumo["Spesa_media_giornaliera_per_100"],
                name="Spesa media (€)",
                mode='lines+markers',
                marker=dict(symbol='square'), # Corrisponde a marker='s'
                line=dict(color='#d62728', width=3) # 'tab:red'
            ),
            secondary_y=True
        )

        # 5. Layout e Stile
        fig.update_layout(
            title_text="<b>Consumo e spesa media giornaliera di sigarette in Italia</b><br>Fonte: ISTAT ed ADM",
            font=dict(family="Arial", size=14),
            template="plotly_white",
            hovermode="x unified", # Mostra entrambi i valori al passaggio del mouse
            legend=dict(x=0.01, y=0.99), # Posizione legenda in alto a sinistra
            margin=dict(r=60) # Margine destro per l'asse secondario
        )

        # Configurazione Asse Y Sinistro (Consumo)
        fig.update_yaxes(
            title_text="Sigarette medie giornaliere (per 100 persone)",
            title_font=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4"),
            secondary_y=False,
            showgrid=True
        )

        # Configurazione Asse Y Destro (Spesa)
        fig.update_yaxes(
            title_text="Spesa media giornaliera (€)",
            title_font=dict(color="#d62728"),
            tickfont=dict(color="#d62728"),
            tickprefix="€",
            secondary_y=True,
            showgrid=False # Griglia solo sull'asse principale per pulizia
        )

        # Configurazione Asse X
        fig.update_xaxes(title_text="Anno", showgrid=True)

        fig.show()

    except Exception as e:
        print(f"Errore: {e}")

generate_dual_axis_chart()